### Tool for Wikipedia

In [17]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [18]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wikipidea_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [19]:
wikipidea_tool.name

'wikipedia'

In [20]:
wikipidea_tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

### Tool for Arxiv

In [21]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv_tool.name

'arxiv'

In [22]:
arxiv_tool.description

'A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.'

### Custom Tool

In [23]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
faiss_db = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = faiss_db.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fa18c830b20>)

In [24]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, 
                                       "langsmith_search",
                                       "Search for inforamtion about LangSmith. For any questions about LangSmith, you must use this tool ")

In [25]:
retriever_tool.name

'langsmith_search'

In [26]:
retriever_tool.description

'Search for inforamtion about LangSmith. For any questions about LangSmith, you must use this tool '

### Combine all tools

In [27]:
tools = [wikipidea_tool, arxiv_tool, retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/opt/anaconda3/envs/chatbot_langchain_env/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for inforamtion about LangSmith. For any questions about LangSmith, you must use this tool ', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7fa184f85750>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain

### LLM

In [29]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)


In [30]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

/opt/anaconda3/envs/chatbot_langchain_env/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

### Agents

In [31]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm=llm,
                                  tools=tools,
                                  prompt=prompt)

#### Agent Executor

In [32]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwc

In [33]:
agent_executor.invoke({"input": "Tell me about LangSmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langsmith2. Create an API key​To create an API key head to the Settings page. Then click Create API Key.3. Set up your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first tr

{'input': 'Tell me about LangSmith',
 'output': 'LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application, enabling you to ship quickly and with confidence. LangSmith works independently, and the use of LangChain is not necessary for its operation.\n\nTo get started with LangSmith, you can follow these steps:\n1. Install LangSmith using Python or TypeScript:\n   - Python: `pip install -U langsmith`\n   - TypeScript: `yarn add langsmith`\n\n2. Create an API key by heading to the Settings page and clicking on "Create API Key."\n\n3. Set up your environment by exporting the necessary variables:\n   ```\n   export LANGCHAIN_TRACING_V2=true\n   export LANGCHAIN_API_KEY=<your-api-key>\n   export OPENAI_API_KEY=<your-openai-api-key>\n   ```\n\n4. Log your first trace to LangSmith. If your application is built entirely on LangChain, you do not need to use the LangSmith SDK directly. You can ref

In [34]:
agent_executor.invoke({"input": "Tell me about Machine Learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine Learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genMachine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generate insights or predictions. It involves creating models that can analyze and make decisions or predictions based on data without being explicitly programmed to do so. Machine learning algorithms are used in various applications such as image recognition, natural language processing, and recommendation systems.

> Finished chain.


{'input': 'Tell me about Machine Learning',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generate insights or predictions. It involves creating models that can analyze and make decisions or predictions based on data without being explicitly programmed to do so. Machine learning algorithms are used in various applications such as image recognition, natural language processing, and recommendation systems.'}

In [35]:
agent_executor.invoke({"input": "How to loose weight with minimal efforts"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'weight loss'}`


Page: Weight loss
Summary: Weight loss, in the context of medicine, health, or physical fitness, refers to a reduction of the total body mass, by a mean loss of fluid, body fat (adipose tissue), or leWeight loss can be achieved through a reduction of total body mass, primarily by losing body fat or fluid. Here are some general tips to help you lose weight with minimal effort:

1. **Healthy Diet**: Focus on eating a balanced and nutritious diet. Include plenty of fruits, vegetables, whole grains, and lean proteins. Avoid processed foods and sugary drinks.

2. **Portion Control**: Be mindful of portion sizes. Eating smaller portions can help you consume fewer calories without feeling deprived.

3. **Stay Hydrated**: Drink plenty of water throughout the day. Sometimes thirst can be mistaken for hunger, leading to unnecessary snacking.

4. **Physical Activity**: Incorporate some form of physica

{'input': 'How to loose weight with minimal efforts',
 'output': "Weight loss can be achieved through a reduction of total body mass, primarily by losing body fat or fluid. Here are some general tips to help you lose weight with minimal effort:\n\n1. **Healthy Diet**: Focus on eating a balanced and nutritious diet. Include plenty of fruits, vegetables, whole grains, and lean proteins. Avoid processed foods and sugary drinks.\n\n2. **Portion Control**: Be mindful of portion sizes. Eating smaller portions can help you consume fewer calories without feeling deprived.\n\n3. **Stay Hydrated**: Drink plenty of water throughout the day. Sometimes thirst can be mistaken for hunger, leading to unnecessary snacking.\n\n4. **Physical Activity**: Incorporate some form of physical activity into your daily routine. This could be as simple as taking a walk, using the stairs instead of the elevator, or doing some light exercises at home.\n\n5. **Get Adequate Sleep**: Lack of sleep can disrupt your met

In [36]:
agent_executor.invoke({"input": "What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloThe paper with the identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. It discusses the study of graphs on lattice points where the edges are derived from a finite set of allocations.

> Finished chain.


{'input': "What's the paper 1605.08386 about?",
 'output': 'The paper with the identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. It discusses the study of graphs on lattice points where the edges are derived from a finite set of allocations.'}